In [1]:
from typing import Any
import numpy as np
import pandas as pd

!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
from pyomo.environ import *

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
D = 3
N = 20
data = []
probTable = None
EG = None
E_ij = {}
strat_ij = {}

# Memoization
memoQ = {}
memoE = {}

Calculating probabilities:

In [3]:
def calculate_q(d, k):
    if (d, k) in memoQ:
        return memoQ[(d, k)]
    if d == 1 and 2 <= k <= 6:
        return 1 / 5

    elif d >= 1 and 2 * d <= k <= 6 * d:
        val = sum([calculate_q(d - 1, k - 2),
                   calculate_q(d - 1, k - 3),
                   calculate_q(d - 1, k - 4),
                   calculate_q(d - 1, k - 5),
                   calculate_q(d - 1, k - 6)]) / 5
        memoQ[(d, k)] = val
        return val
    return 0

def calculate_prob(d, k):
    if k == 1:
        return 1 - pow((5 / 6), d)
    elif 2 <= k <= 2 * d - 1 or k > 6 * d:
        return 0
    else:
        return pow((5 / 6), d) * calculate_q(d, k)

def generate_prob_table(D, K):
    dict_prob = np.zeros((D + 1, K + 1))
    # useless
    dict_prob[0, 0] = 1
    for i in range(1, D + 1):
        for j in range(1, K + 1):
            dict_prob[i, j] = round(calculate_prob(i, j), 6)

    return dict_prob

First Part : One Tour Game

In [4]:
def calculate_EG_d1_d2(d1, d2):
    p1 = 0
    p2 = 0

    print('Calculating Gain For Case (', d1, ',', d2, ')...')
    for i in range(1, 6 * d1 + 1):
        for j in range(1, 6 * d2 + 1):
            if j < i:
                p1 += probTable[d1, i] * probTable[d2, j]
            elif j > i:
                p2 += probTable[d1, i] * probTable[d2, j]

    print('Gain For Case (', d1, ',', d2, ') Estimated With ', p1 - p2)
    return round(p1 - p2, 5)

#print('Generating Probability Table...')
#probTable = generate_prob_table(D, 6 * D)
#print('Probability Table Generated.')

Generating gain matrix EG:

In [5]:
def generate_EG():
    print('Generating Probability Table...')
    print('Probability Table Generated.')

    for i in range(D + 1):
        subData = []
        for j in range(D + 1):
            subData.append(calculate_EG_d1_d2(i, j))
        data.append(subData)

    print('Generating Gain Matrix...')
    arr = np.array(data)
    print('Gain Matrix Generated.')
    print(arr)

    return arr

Second Part : Repetitive Game

In [6]:
print('Generating Probability Table...')
probTable = generate_prob_table(D, N + 6 * D)
print('Probability Table Generated.')

Generating Probability Table...
Probability Table Generated.


Finding the optimal strategy for player 1's linear program:

In [7]:
# Finding pl1's mixed strategy using MaxiMin
def solve_linear_pl1(G):
    model = ConcreteModel()
    model.alpha = Var(bounds=(-np.inf, np.inf))

    n = G.shape[0] - 1
    model.I = RangeSet(1, n)
    model.P = Var(model.I, bounds=(0, 1))

    model.C = ConstraintList()
    for j in range(1, G.shape[1]):
        model.C.add(model.alpha <= np.array([G[i, j] * model.P[i] for i in model.I]).sum())

    model.final_C = Constraint(
        expr=sum(model.P[i] for i in model.I) == 1
        )

    # add the objective
    model.obj = Objective(expr=model.alpha, sense=maximize)
    print(model)
    SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

    return model.alpha(), [model.P[i]() for i in model.I]

In case player 2 knows about player 1's distributed probability vector for optimal strategy, then he might try to find his best response in pure strategies:

In [8]:
# Player 2 knows Pl 1's prob vector, so here we look for Pl 2's optimal pure strategy
# in other words, we minimize his loss
def find_pure_strategy(P1, G):
    if G.shape and P1 is not None:
        G = G[1:, 1:]
        res = np.array([(G[:, j] * P1).sum() for j in range(0, G.shape[1])])
        print(res)
        return res.argmin() + 1

In [9]:
def init_EG():
    EG = np.zeros((N + 6 * D, N + 6 * D))

    for i in range(N, N + 6 * D):
        for a in range(0, N - 1):
            EG[i, a] = 1
            EG[a, i] = -1
    for i in range(N, N + 6 * D):
        for j in range(N, N + 6 * D):
            if i > j:
                EG[i, j] = 1
                EG[j, i] = -1
            elif j > i:
                EG[j, i] = 1
                EG[i, j] = -1
            else:
                EG[i, j] = 0
    return EG

def calculate_E_ij_d1_d2(d1, d2, i, j):
    if (i, j, d1, d2) in memoE.keys():
        return memoE[(i, j, d1, d2)] 
    if i >= N or j >= N:
        return EG[i, j]

    if i < N and j < N:
        P1 = 0
        P2 = 0
        P3 = 0
        # Chances that score 1 might be bigger than the objective N and score 2
        for k in range(N - i, N + 6 * d1):
            for l in range(1, k):
                if k <= 6 * d1 and l <= 6 * d2:
                    P1 += probTable[d1, k] * probTable[d2, l]

            # Chances that score 2 might be bigger than the objective N and score 1
        for k in range(N - j, N + 6 * d2):
            for l in range(1, k):
                if k <= 6 * d2 and l <= 6 * d1:
                    P2 += probTable[d1, l] * probTable[d2, k]

        # Chances that score 1 and score 2 are smaller then the objective SO we attach the next tour's gain
        for k in range(1, N - i):
            for l in range(1, N - j):
                if k <= 6 * d1 and l <= 6 * d2:
                    P3 += probTable[d1, k] * probTable[d2, l] + \
                          calculate_EG_k_l(i + k, j + l)

        val = P1 - P2 + P3
        memoE[(i, j, d1, d2)] = val
        return val

def calculate_EG_k_l(k, l):
    if k >= N or l >= N:
        return EG[k, l]
    if (k, l) not in E_ij.keys():
        calculate_E_ij(k, l)

    x, y = solve_linear_pl1(E_ij[(k, l)])
    EG[k, l] = x
    strat_ij[(k, l)] = y[1:]

    return EG[k, l]

def calculate_E_ij(i, j):
    E = np.zeros((D + 1, D + 1))
    for d1 in range(1, D + 1):
        for d2 in range(1, D + 1):
            E[d1, d2] = calculate_E_ij_d1_d2(d1, d2, i, j)

    E_ij[(i, j)] = E

Generating gain matrix EG that contains the optimal strategy for each state (i, j):

In [10]:
EG = init_EG()

def generate_full_EG():
    calculate_E_ij(0, 0)
    print('E_ij\n')
    print(E_ij)
    print('EG\n')
    print(EG)
    print('Strategy_ij\n')
    print(strat_ij)

Test:

In [11]:
D = 3
N = 20
#gainM = generate_EG()
#alpha, p = solve_linear_pl1(gainM)
#print(alpha, p)
#print(find_pure_strategy(p, gainM))
generate_full_EG()
'''df = pd.DataFrame(EG)
print('Generating HTML...')
f = open("EG.html", "w")
f.write(df.to_html())
print('HTML Generated.')'''

unknown
    executable for solver cbc. File with name=/usr/bin/cbc either does not
    exist or it is not executable. To skip this validation, call
    set_executable with validate=False.
    unavailable solver.

    The SolverFactory was unable to create the solver "_cbc_shell" and
    returned an UnknownSolver object.  This error is raised at the point where
    the UnknownSolver object was used as if it were valid (by calling method
    "set_problem_format").

    The original solver was created with the following parameters:
    	executable: /usr/bin/cbc type: _cbc_shell _args: () options: {}


RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "cbc"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: /usr/bin/cbc
	type: cbc
	_args: ()
	options: {}